<a href="https://colab.research.google.com/github/Carol-P18/-Identification-of-Complementarity-determining-region3-CDRH3-on-the-heavy-chain-using-a-logistic-re/blob/main/Copy_of_B2_Pandas_and_SQL_Inserting_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Minimal Example of Loading a Dataset to a Database

In [1]:
!pip3 install -U -q PyMySQL sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.8 MB/s eta 0:00:00


In [2]:
import pandas as pd

In [3]:
from sqlalchemy import create_engine

## Downloading Data and Putting in a Dataframe

In [4]:
!curl 'https://data.cityofnewyork.us/api/views/43nn-pn8j/rows.csv?accessType=DOWNLOAD' -o restaurants.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 98.5M    0 98.5M    0     0  6110k      0 --:--:--  0:00:16 --:--:-- 6732k


In [5]:
# Read the CSV file
df = pd.read_csv('restaurants.csv')

In [6]:
# Adding underscores in all column names
cols = df.columns
cols = cols.map(lambda x: x.replace(' ', '_').upper())
df.columns = cols

In [7]:
cols

Index(['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE_DESCRIPTION', 'INSPECTION_DATE', 'ACTION', 'VIOLATION_CODE',
       'VIOLATION_DESCRIPTION', 'CRITICAL_FLAG', 'SCORE', 'GRADE',
       'GRADE_DATE', 'RECORD_DATE', 'INSPECTION_TYPE', 'LATITUDE', 'LONGITUDE',
       'COMMUNITY_BOARD', 'COUNCIL_DISTRICT', 'CENSUS_TRACT', 'BIN', 'BBL',
       'NTA', 'LOCATION_POINT1'],
      dtype='object')

In [27]:
# Some bookkeeping regarding datatypes
df["INSPECTION_DATE"] = pd.to_datetime(df["INSPECTION_DATE"], format="%m/%d/%Y")
df["SCORE"] = pd.to_numeric(df["SCORE"])

# Delete useless columns
df = df.drop(['GRADE_DATE', 'RECORD_DATE', 'LOCATION_POINT1'], axis='columns')

## Create MySQL Connection

In [15]:
import os
from sqlalchemy import create_engine, text

# Connect to the MySQL, and use the "public" database
conn_string = 'mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8'.format(
    user     = 'student',
    password = 'dwdstudent2015',
    host     = 'db.ipeirotis.org',
    port     = 3306,
    encoding = 'utf-8',
    db = 'public' #this is a databsae that is writable by a student account so you can insert data into it
)
engine = create_engine(conn_string)

## Create the table for storing the data

Although we can let Pandas create the table automatically, the choice of data types of not always great. It is better to manually define the data types for the database.

In [16]:
# Report the maximum string lengths for
# the textual attributes. Useful when creating
# a table in SQL.
for c in df.columns.values:
  if df.dtypes[c] == 'object':
    print(c, df[c].str.len().max())


DBA 100.0
BORO 13
BUILDING 10.0
STREET 40.0
PHONE 12.0
CUISINE_DESCRIPTION 30.0
INSPECTION_DATE 10
ACTION 130.0
VIOLATION_CODE 5.0
VIOLATION_DESCRIPTION 952.0
CRITICAL_FLAG 14
GRADE 1.0
GRADE_DATE 10.0
RECORD_DATE 10
INSPECTION_TYPE 59.0
NTA 4.0


In [18]:
# To avoid conflicts between people writing in the same database, we add a random suffix in the tables
# We only create the variable once while running the notebook
import uuid
if "suffix" not in globals():
    suffix = str(uuid.uuid4())[:8]
print(suffix)

e5d7450e


In [21]:
# Define the MySQL database that we will use to store the table
db_name = "public"

# The name f the table that we will use
table_name = f"inspections_{suffix}"
table_name

'inspections_e5d7450e'

In [25]:
#creates the tables with the datatypes/variables etc
drop_table_sql = f'''
DROP TABLE IF EXISTS {db_name}.{table_name};
'''

with engine.connect() as con:
    con.execute(text(drop_table_sql))


create_table_sql = f'''
CREATE TABLE IF NOT EXISTS {db_name}.{table_name} (
    CAMIS CHAR(8),
    DBA VARCHAR(100),
    BUILDING VARCHAR(10),
    STREET VARCHAR(40),
    ZIPCODE CHAR(5),
    BORO VARCHAR(15),
    PHONE CHAR(12),
    CUISINE_DESCRIPTION VARCHAR(30),
    LATITUDE FLOAT,
    LONGITUDE FLOAT,
    COMMUNITY_BOARD CHAR(3),
    COUNCIL_DISTRICT CHAR(2),
    CENSUS_TRACT CHAR(6),
    BIN CHAR(7),
    BBL CHAR(10),
    NTA CHAR(4),
    INSPECTION_DATE DATETIME,
    ACTION VARCHAR(130),
    GRADE CHAR(1),
    INSPECTION_TYPE VARCHAR(60),
    VIOLATION_CODE VARCHAR(10),
    VIOLATION_DESCRIPTION VARCHAR(1000),
    CRITICAL_FLAG VARCHAR(15),
    SCORE SMALLINT
)  ENGINE=INNODB DEFAULT CHARSET=UTF8MB4;
'''

with engine.connect() as con:
    con.execute(text(create_table_sql))



## Insert data to DB using the `to_sql` command

In [28]:
# Store the dataframe as a SQL table, using the to_sql command
df.to_sql(name=table_name, # name of the table &data frame assigned as df
                   con=engine, # use the connection to MySQL created earlier
                   if_exists='append', # we created the empty table above
                   index=False, # do not write the index column in the database
                   chunksize=1000 # write 1000 lines at a time
)
#Returns the number of rows = 232943

232943

In [ ]:
# And then we can just retrieve it from the database
with engine.connect() as connection:
  r = pd.read_sql(text(f"SELECT * FROM public.{table_name} LIMIT 100"), con=connection)
r.head(100)